<a href="https://colab.research.google.com/github/SeongwonTak/RecommenderSystem/blob/main/Basic_code_Recom_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender System 기초 구현

## 기본 준비 및 전처리

https://lsjsj92.tistory.com/565 를 따라하며 구현을 실습한다.

물론, 거기에 직접 일부 부분 / 추가로 실험하고 싶은 부분은 추가 실험을 해 보기로 한다.

예를 들어, neighborhood를 확인할 때, 다른 metric을 부여한다던가.

사용하는 Dataset은 영화 관련 Dataset이다.
https://www.kaggle.com/rounakbanik/the-movies-dataset

In [1]:
# 패키지를 먼저 import 한다. 추가로 필요한 패키지는 그때그때.
import numpy as np
import pandas as pd
import sklearn as sk

In [57]:
# Step 0. 데이터를 불러온다.
movie_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RecoExample/movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
movie_data.tail(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,0.163015,/s5UkZt6NTsrS7ZF0Rh8nzupRlIU.jpg,[],"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


우선은 영화의 Meta data들부터 분석하려고 한다.
여러 변수들이 있는데, 사용할 변수를 추리기 위해 우선은 변수들을 정리한다.
- id : 영화 번호
- title : 영화 제목
- genres : 영화 장르
- popularity : 영화 인기도
- vote_average : 평균 평점
- vote_count : 평점 매긴 사람의 수
- release_date : 영화 발매일
- adult : 말 그대로, 성인영화인가?
- overview : 개요

이정도만 사용하려고 한다.

In [59]:
movie_data = movie_data[['id','title', 'genres', 'popularity', 'vote_average', 'vote_count', 'release_date', 'adult', 'overview']]

각 항목들에 대해서 전처리를 실시해야 한다.

In [60]:
movie_data.tail(5)

,id,title,genres,popularity,vote_average,vote_count,release_date,adult,overview
45461,439050,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",0.072051,4.0,1.0,NaN,False,Rising and falling between a man and woman.
45462,111109,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",0.178241,9.0,3.0,2011-11-17,False,An artist struggles to finish his work while a...
45463,67758,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",0.903007,3.8,6.0,2003-08-01,False,"When one of her hits goes wrong, a professiona..."
45464,227506,Satan Triumphant,[],0.003503,0.0,0.0,1917-10-21,False,"In a small town live two brothers, one a minis..."
45465,461257,Queerama,[],0.163015,0.0,0.0,2017-06-09,False,50 years after decriminalisation of homosexual...


장르를 의미있게 활용하기 위해,keyword를 별도로 부른 후 keyword를 movie_data와 연결하려고 한다.

In [61]:
keywords = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RecoExample/keywords.csv')

In [62]:
keywords.tail()

,id,keywords
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]
46418,461257,[]


In [63]:
# 타입을 통일시켜서 붙일 것이다.
# 후에 문자열 전처리 도구를 사용할것이기에, string 변경.
movie_data.id = movie_data.id.astype(str)
keywords.id = keywords.id.astype(str)
movie_combined = pd.merge(movie_data, keywords)

In [64]:
movie_combined.tail()

,id,title,genres,popularity,vote_average,vote_count,release_date,adult,overview,keywords
46477,439050,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",0.072051,4.0,1.0,NaN,False,Rising and falling between a man and woman.,"[{'id': 10703, 'name': 'tragic love'}]"
46478,111109,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",0.178241,9.0,3.0,2011-11-17,False,An artist struggles to finish his work while a...,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46479,67758,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",0.903007,3.8,6.0,2003-08-01,False,"When one of her hits goes wrong, a professiona...",[]
46480,227506,Satan Triumphant,[],0.003503,0.0,0.0,1917-10-21,False,"In a small town live two brothers, one a minis...",[]
46481,461257,Queerama,[],0.163015,0.0,0.0,2017-06-09,False,50 years after decriminalisation of homosexual...,[]


우선 먼저 결측치부터 정리하려고 한다.

In [ ]:
movie_combined.isnull().sum()

id                0
genres            0
popularity        4
vote_average      4
vote_count        4
release_date     88
adult             0
overview        995
keywords          0
dtype: int64

In [65]:
movie_combined = movie_combined.dropna(axis=0, how='any')

In [ ]:
movie_combined.tail()

,id,genres,popularity,vote_average,vote_count,release_date,adult,overview,keywords
46476,30840,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",5.683753,5.7,26.0,1991-05-13,False,"Yet another version of the classic epic, with ...",[]
46478,111109,"[{'id': 18, 'name': 'Drama'}]",0.178241,9.0,3.0,2011-11-17,False,An artist struggles to finish his work while a...,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46479,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",0.903007,3.8,6.0,2003-08-01,False,"When one of her hits goes wrong, a professiona...",[]
46480,227506,[],0.003503,0.0,0.0,1917-10-21,False,"In a small town live two brothers, one a minis...",[]
46481,461257,[],0.163015,0.0,0.0,2017-06-09,False,50 years after decriminalisation of homosexual...,[]


Index가 꼬여 있으면 영 좋지 않다. index를 정리해주자.

In [66]:
movie_combined.reset_index(inplace = True, drop = True)

In [ ]:
movie_combined.tail()

,id,genres,popularity,vote_average,vote_count,release_date,adult,overview,keywords
45407,30840,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",5.683753,5.7,26.0,1991-05-13,False,"Yet another version of the classic epic, with ...",[]
45408,111109,"[{'id': 18, 'name': 'Drama'}]",0.178241,9.0,3.0,2011-11-17,False,An artist struggles to finish his work while a...,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
45409,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",0.903007,3.8,6.0,2003-08-01,False,"When one of her hits goes wrong, a professiona...",[]
45410,227506,[],0.003503,0.0,0.0,1917-10-21,False,"In a small town live two brothers, one a minis...",[]
45411,461257,[],0.163015,0.0,0.0,2017-06-09,False,50 years after decriminalisation of homosexual...,[]


또한 다음 데이터들도 버려야 할 것이다.
- genres가 없는 데이터들
- vote_count가 0인 것들(... 왜 있어?)
- keywords가 없는 것들

In [67]:
# 사고대비
movie_backup = movie_combined.copy()

In [ ]:
movie_backup.dtypes

id               object
genres           object
popularity       object
vote_average    float64
vote_count      float64
release_date     object
adult            object
overview         object
keywords         object
dtype: object

In [68]:
movie_backup = movie_backup.loc[movie_backup['vote_count']>0,:]

In [69]:
movie_backup = movie_backup.loc[movie_backup['keywords'] != '[]',:]
movie_backup = movie_backup.loc[movie_backup['genres'] != '[]',:]

In [11]:
movie_backup

,id,genres,popularity,vote_average,vote_count,release_date,adult,overview,keywords
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.9469,7.7,5415.0,1995-10-30,False,"Led by Woody, Andy's toys live happily in his ...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.0155,6.9,2413.0,1995-12-15,False,When siblings Judy and Peter discover an encha...,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",11.7129,6.5,92.0,1995-12-22,False,A family wedding reignites the ancient feud be...,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.85949,6.1,34.0,1995-12-22,False,"Cheated on, mistreated and stepped on, the wom...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",8.38752,5.7,173.0,1995-02-10,False,Just when George Banks has recovered from his ...,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...
45400,404604,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",1.559596,6.6,14.0,2017-07-07,False,The bliss of a biology teacher’s family life i...,"[{'id': 155794, 'name': 'physical abuse'}, {'i..."
45403,84419,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",0.222814,6.3,8.0,1946-03-29,False,An unsuccessful sculptor saves a madman named ...,"[{'id': 9748, 'name': 'revenge'}, {'id': 9826,..."
45404,390959,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",0.076061,7.0,2.0,2000-10-22,False,"In this true-crime documentary, we delve into ...","[{'id': 224180, 'name': 'blair witch'}]"
45405,289923,"[{'id': 27, 'name': 'Horror'}]",0.38645,7.0,1.0,2000-10-03,False,A film archivist revisits the story of Rustin ...,"[{'id': 616, 'name': 'witch'}, {'id': 2035, 'n..."


In [70]:
movie_combined = movie_backup
movie_combined.reset_index(inplace = True, drop = True)

위의 과정에서 부딪힌 문제 상황은 다음과 같다
- genres와 keywords를 list화 시켜야 한다.
ast 패키지의 literal_eval을 활용할 수 있다.

In [71]:
import ast
movie_combined['genres'] = movie_combined['genres'].apply(ast.literal_eval)

In [72]:
movie_combined['keywords'] = movie_combined['keywords'].apply(ast.literal_eval)

이제 구조를 자세히 보면, list내 각 dict의 name에 있는 것을 다시 list화 시켜야 함을 알 수 있다.

In [73]:
movie_combined['genres'] = movie_combined['genres'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))

In [74]:
movie_combined['keywords'] = movie_combined['keywords'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))

In [17]:
movie_combined

,id,genres,popularity,vote_average,vote_count,release_date,adult,overview,keywords
0,862,Animation Comedy Family,21.9469,7.7,5415.0,1995-10-30,False,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...
1,8844,Adventure Fantasy Family,17.0155,6.9,2413.0,1995-12-15,False,When siblings Judy and Peter discover an encha...,board game disappearance based on children's b...
2,15602,Romance Comedy,11.7129,6.5,92.0,1995-12-22,False,A family wedding reignites the ancient feud be...,fishing best friend duringcreditsstinger old men
3,31357,Comedy Drama Romance,3.85949,6.1,34.0,1995-12-22,False,"Cheated on, mistreated and stepped on, the wom...",based on novel interracial relationship single...
4,11862,Comedy,8.38752,5.7,173.0,1995-02-10,False,Just when George Banks has recovered from his ...,baby midlife crisis confidence aging daughter ...
...,...,...,...,...,...,...,...,...,...
29897,404604,Crime Drama Thriller,1.559596,6.6,14.0,2017-07-07,False,The bliss of a biology teacher’s family life i...,physical abuse sexual assault
29898,84419,Horror Mystery Thriller,0.222814,6.3,8.0,1946-03-29,False,An unsuccessful sculptor saves a madman named ...,revenge murder serial killer new york city scu...
29899,390959,Mystery Horror,0.076061,7.0,2.0,2000-10-22,False,"In this true-crime documentary, we delve into ...",blair witch
29900,289923,Horror,0.38645,7.0,1.0,2000-10-03,False,A film archivist revisits the story of Rustin ...,witch mythology legend serial killer mockumentary


다음은 평점에 대해 전처리를 실시하려고 한다.
평점에 대한 전처리를 하는 이유는 인원수에 대한 편차를 줄이기 위함이다.
https://www.quora.com/How-does-IMDbs-rating-system-work

를 기준으로 하여 평점을 변경해보자.

- R : 특정 영화의 평균 평점
- V : 특정 영화의 투표 수
- C : 전체 영화의 평균 평점
- M : 특정 등수(여기서는 약 상위 10%인 3000위정도로 하자.)에 들기 위한 최소 요구 투표수.

산식은

$$ \frac{V}{V+M} \times R + \frac{M}{V+M} \times C $$ 으로 주어진다.

각 값들을 구해보자.

In [75]:
# C 구하기.
C = movie_combined['vote_average'].mean()
C

6.052330947762581

In [76]:
# M 구하기
# 0.9분위수로 충분하다
M = movie_combined['vote_count'].quantile(0.9)
M

302.0

In [77]:
def weight_rate(x, m=M, c=C):
  # 행별로 들어갈 함수
  V = x['vote_count']
  R = x['vote_average']

  return (V / (V + m) * R) + (m / (m + V) * c)

In [78]:
movie_combined['adj_score'] = movie_combined.apply(weight_rate, axis = 1)

In [98]:
movie_combined.head(10)

,id,title,genres,popularity,vote_average,vote_count,release_date,adult,overview,keywords,adj_score
0,862,Toy Story,Animation Comedy Family,21.9469,7.7,5415.0,1995-10-30,False,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,7.612962
1,8844,Jumanji,Adventure Fantasy Family,17.0155,6.9,2413.0,1995-12-15,False,When siblings Judy and Peter discover an encha...,board game disappearance based on children's b...,6.805710
2,15602,Grumpier Old Men,Romance Comedy,11.7129,6.5,92.0,1995-12-22,False,A family wedding reignites the ancient feud be...,fishing best friend duringcreditsstinger old men,6.156863
3,31357,Waiting to Exhale,Comedy Drama Romance,3.85949,6.1,34.0,1995-12-22,False,"Cheated on, mistreated and stepped on, the wom...",based on novel interracial relationship single...,6.057155
4,11862,Father of the Bride Part II,Comedy,8.38752,5.7,173.0,1995-02-10,False,Just when George Banks has recovered from his ...,baby midlife crisis confidence aging daughter ...,5.924008
5,949,Heat,Action Crime Drama Thriller,17.9249,7.7,1886.0,1995-12-15,False,"Obsessive master thief, Neil McCauley leads a ...",robbery detective bank obsession chase shootin...,7.472580
6,11860,Sabrina,Comedy Romance,6.67728,6.2,141.0,1995-12-15,False,An ugly duckling having undergone a remarkable...,paris brother brother relationship chauffeur l...,6.099332
7,9091,Sudden Death,Action Adventure Thriller,5.23158,5.5,174.0,1995-12-22,False,International action superstar Jean Claude Van...,terrorist hostage explosive vice president,5.850428
8,710,GoldenEye,Adventure Action Thriller,14.686,6.6,1194.0,1995-11-16,False,James Bond must unmask the mysterious head of ...,cuba falsely accused secret identity computer ...,6.489441
9,9087,The American President,Comedy Drama Romance,6.31844,6.5,199.0,1995-11-17,False,"Widowed U.S. president Andrew Shepherd, one of...",white house usa president new love widower wil...,6.230148


## Content-based Model의 구현

가장 먼저 할 일은 Content-based Model에 대한 구현이다.
장르 기준으로 추천 모델을 짜보려고 한다.
- 유사도가 높을수록 먼저 추천
- 동일한 유사도일 경우 Adj_score가 더 높은 아이템을 추천한다.

먼저 장르가 현재 어떻게 생겨먹었나를 보자.

In [27]:
movie_combined.genres.head(10)

0        Animation Comedy Family
1       Adventure Fantasy Family
2                 Romance Comedy
3           Comedy Drama Romance
4                         Comedy
5    Action Crime Drama Thriller
6                 Comedy Romance
7      Action Adventure Thriller
8      Adventure Action Thriller
9           Comedy Drama Romance
Name: genres, dtype: object

우리가 원하는 짓은 여기 있는 단어들을 하나하나 열로 만들어서 vector화 시키고 싶은 것이다. 

문제중 하나는 저 많은 영화 데이터에 얼마나 많은 장르가 담겨있는지 알기가 어렵고, 길이도 제마다 달라 추출이 어렵다. 집합을 사용하여 하나하나 빼는 방법도 있겠으나, 다른 방법을 써보자.

### CountVectorizer

Scikit-learn에 있는 기능을 하나 가져와보자.
하려는 행동은, 각 줄에 들어가있는 단어들이 몇 개인지 세는 것이다. 

즉, movie-genre matrix의 생성이다.
**즉 어떤 영화가 특정 장르에 속하면 1, 아니면 0을 반환하는 행렬을 만들어주면 된다.**
이를 CountVectorzier를 통해 생성해 줄 수 있다.


In [79]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
cnt_genres = count_vector.fit_transform(movie_combined['genres'])
genres_matrix = cnt_genres.toarray()

In [80]:
print(genres_matrix)

[[0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [81]:
#column별 의미 추출
print(count_vector.vocabulary_)

{'animation': 2, 'comedy': 3, 'family': 7, 'adventure': 1, 'fantasy': 8, 'romance': 16, 'drama': 6, 'action': 0, 'crime': 4, 'thriller': 18, 'horror': 12, 'history': 11, 'science': 17, 'fiction': 9, 'mystery': 15, 'war': 20, 'foreign': 10, 'music': 14, 'documentary': 5, 'western': 21, 'tv': 19, 'movie': 13}


### Similarity
이제 위의 결과물로 유사도를 활용하여 추천 모델을 만들어보자.



목표는 다음과 같다. 

1. 영화-영화간의 유사도를 구하자.
2. 특정 영화 제목이 주어질 때, 유사도가 가장 높은 영화 10개를 선정한다.

우리가 사용할 유사도는 다양한 것들 중 cosine similarity를 사용할려고 한다.
말 그대로, cosine값을 주려고 한다.

In [82]:
def cosine_similarity(a, b):
  norm_a = np.linalg.norm(a, ord = 2)
  norm_b = np.linalg.norm(b, ord = 2)
  return np.dot(a, b) / (norm_a * norm_b)
# 사실은 scikit-learn에 패키지가 있으나, 직접 만들었다.

In [83]:
genres_matrix.shape[0]

29902

즉 29,902개의 영화 중 제목이 일치하는 것을 하나 잡은 후, 제목이 일치하는 인덱스를 뽑아야 한다.

In [99]:
target_title = str(input())
target_index = movie_combined[movie_combined['title'] == target_title].index.values[0]
print(target_index)
target_sim = [0 for _ in range(29902)]
for i in range(29902):
  if i == target_index:
    target_sim[i] = 0
  else:
    target_sim[i] = cosine_similarity(genres_matrix[i], genres_matrix[target_index])

result_sim_df = pd.DataFrame(data = target_sim, index = range(29902), columns=['sim'])
result = pd.concat([movie_combined, result_sim_df], axis = 1)
final_result = result.sort_values('sim', ascending=False)[:10]['title']
print(final_result)

Jumanji
1
10535                       The Spiderwick Chronicles
25714                      Kingdom of Crooked Mirrors
1696                                     Return to Oz
13280    Harry Potter and the Deathly Hallows: Part 1
21104                                             Pan
28242                                            Mama
26786                                   Pete's Dragon
22023         Fantastic Beasts and Where to Find Them
12400                             Alice in Wonderland
28158                     Adventures in Dinosaur City
Name: title, dtype: object


다만, 이렇게 설계할 경우 아쉬운 점은 실제 영화의 평점 등은 전혀 반영되지 않았다는 점이다. 오로지 '장르의 유사성'에만 집중하여 영화의 추천이 일어났다.
다만, 이번 구현에서는 가장 원론적인 구현 방법론에만 집중하려 한다.

## Collaborative-Filtering의 구현

Matrix Factorization 이전에, 